In [157]:
import os 
import json
import pandas as pd
import traceback

In [158]:
from langchain_openai import ChatOpenAI

In [159]:
from dotenv import load_dotenv
load_dotenv() # take enviroment variable from .env

True

In [160]:
mykey = os.getenv("OPENAI_API_KEY")

In [161]:
llm = ChatOpenAI(openai_api_key = mykey, model = 'gpt-4o', temperature=0.2)

In [162]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002D605009EB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002D604FF4550>, root_client=<openai.OpenAI object at 0x000002D603F92940>, root_async_client=<openai.AsyncOpenAI object at 0x000002D605009EE0>, model_name='gpt-4o', temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='')

In [163]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [164]:
RESPONSE_JSON = {
    '1': {
        'mcq':'multiple choices question',
        'options': {
            'a':"choice here",
            'b':"choice here",
            'c':"choice here",
            'd':"choice here",
        },
        'correct':"correct answer"
    },
    '2':{
        'mcq':"multiple choices question",
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
    },
    '3':{
        'mcq':'multiple choices question',
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
    },
    '4':{
        'mcq':'multiple choices question',
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
}
}

In [165]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs and Return only JSON, without any additional text or Markdown code blocks (```).
### RESPONSE_JSON
{response_json}

"""

In [166]:
quiz_generation_prompt = PromptTemplate(
    input_variables= ['text', 'number', 'subject', 'tone','response_json'],
    template = TEMPLATE
)

In [167]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [168]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [169]:
quiz_evaluation_prompt = PromptTemplate(input_variables=['subject','quiz'],
                                        template= TEMPLATE2)

In [170]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key = 'review', verbose = True)

In [171]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [172]:
file_path = r"C:\Users\LEX\mcqgen\data.txt"

In [173]:
with open(file_path, 'r') as file:
    TEXT = file.read()

TEXT = TEXT.lstrip("\ufeff")
print(TEXT)

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within AI, leading to inductive logic 

In [174]:
# Dic into Json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choices question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}}'

In [175]:
NUMBER = 5,
SUBJECT = "machine learning"
TONE = "simple"

In [176]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
        
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{"1": {"mcq": "What was the original goal of machine learning as a scientific endeavor?", "options": {"a": "To develop expert systems", "b": "To achieve artificial intelligence", "c": "To improve data storage", "d": "To enhance computer graphics"}, "correct": "b"}, "2": {"mcq": "Which approach caused a rift between AI and machine learning?", "options": {"a": "Neural networks", "b": "Probabilistic reasoning", "c": "Logical, knowledge-based approach", "d": "Pattern recognition"}, 

In [177]:
print(cb.total_tokens)
print(cb.prompt_tokens)
print(cb.completion_tokens)
print(cb.total_cost)

1894
1158
736
0.010255


In [178]:
print(response)

{'text': 'As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]:\u200a488\u200a\n\nHowever, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]:\u200a488\u200a By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within

In [179]:
quiz = response.get('quiz')
type(quiz)

str

In [180]:
print(f"quiz 的长度: {len(quiz)}")
print(f"quiz 内容（前100个字符）: {quiz[:100]}")


quiz 的长度: 1230
quiz 内容（前100个字符）: {"1": {"mcq": "What was the original goal of machine learning as a scientific endeavor?", "options":


In [181]:
quiz  = json.loads(quiz)

In [182]:
print(quiz)

{'1': {'mcq': 'What was the original goal of machine learning as a scientific endeavor?', 'options': {'a': 'To develop expert systems', 'b': 'To achieve artificial intelligence', 'c': 'To improve data storage', 'd': 'To enhance computer graphics'}, 'correct': 'b'}, '2': {'mcq': 'Which approach caused a rift between AI and machine learning?', 'options': {'a': 'Neural networks', 'b': 'Probabilistic reasoning', 'c': 'Logical, knowledge-based approach', 'd': 'Pattern recognition'}, 'correct': 'c'}, '3': {'mcq': 'What was the main success of researchers like John Hopfield and Geoffrey Hinton in the mid-1980s?', 'options': {'a': 'Development of expert systems', 'b': 'Invention of neural networks', 'c': 'Reinvention of backpropagation', 'd': 'Creation of fuzzy logic'}, 'correct': 'c'}, '4': {'mcq': 'In which decade did machine learning start to flourish as its own field?', 'options': {'a': '1970s', 'b': '1980s', 'c': '1990s', 'd': '2000s'}, 'correct': 'c'}, '5': {'mcq': 'What did machine lear

In [183]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [184]:
quiz_table_data

[{'MCQ': 'What was the original goal of machine learning as a scientific endeavor?',
  'Choices': 'a: To develop expert systems | b: To achieve artificial intelligence | c: To improve data storage | d: To enhance computer graphics',
  'Correct': 'b'},
 {'MCQ': 'Which approach caused a rift between AI and machine learning?',
  'Choices': 'a: Neural networks | b: Probabilistic reasoning | c: Logical, knowledge-based approach | d: Pattern recognition',
  'Correct': 'c'},
 {'MCQ': 'What was the main success of researchers like John Hopfield and Geoffrey Hinton in the mid-1980s?',
  'Choices': 'a: Development of expert systems | b: Invention of neural networks | c: Reinvention of backpropagation | d: Creation of fuzzy logic',
  'Correct': 'c'},
 {'MCQ': 'In which decade did machine learning start to flourish as its own field?',
  'Choices': 'a: 1970s | b: 1980s | c: 1990s | d: 2000s',
  'Correct': 'c'},
 {'MCQ': 'What did machine learning shift its focus towards in the 1990s?',
  'Choices':

In [185]:
quiz  = pd.DataFrame(quiz_table_data)

In [186]:
quiz.to_csv("machine learning.csv", index=True)

In [187]:
from datetime import datetime
datetime.now()

datetime.datetime(2025, 2, 26, 21, 28, 37, 169533)